In [2]:
%load_ext autoreload
%autoreload 2
# %pylab inline
import pandas
import os
os.sys.path.append('../')

(spk-sent ID) (utterance01,utterance02,utterance03)

## RedDots Dataset

The RedDots project was initiated, with collaboration from multiple sites, as a follow-up to a special session during INTERSPEECH 2014. It was set out to collect speech data through mobile crowd-sourcing, with the benefit of potentially wider population and greater diversity. The project was rolled out in January 29, 2015.The current RedDots_r2015q4_v1 release contains all the audited recordings up to August 17th 2015. It has 62 speakers including 49 male speakers and 13 female speakers from 21 countries. The total number of sessions for the current release is 572 (473 male and 99 female sessions).

__File format__

* Enrol (.trn)
    * (spk-sent ID) (utterance01,utterance02,utterance03)
    * e.g.: m0001_31 m0001/20150130084154554_m0001_31,m0001/20150130084155412_m0001_31,m0001/20150130084156114_m0001_31

* Trial (.ndx):
    * (spk-sent ID),(test utterance),(is target-correct),(is target-wrong),(is imposter-correct),(is imposter-wrong),
    * e.g.: m0001_31,m0001/20150129213253016_m0001_36,N,Y,N,N
    
    
<hr>    

|                     |  Male |  Female |  Remark        |
|---------------------|-------|---------|----------------|
|  # target speakers  |  35   |  6      |  >= 6 sessions |
|  # unseen imposters |  14   |  7      |  < 6 sessions  |
|  total              |  49   |  13     |  -             |


In [14]:
rDots_home_dir = '/home/muncok/DL/dataset/SV_sets/reddots_r2015q4_v1/'
rDots_pcm_dir = os.path.join(rDots_home_dir, 'pcm/')
rDots_wav_dir = os.path.join(rDots_home_dir, 'wav/')
rDots_ndx_dir = os.path.join(rDots_home_dir, 'ndx/')
rDots_infos_dir = os.path.join(rDots_home_dir, 'infos/')

In [15]:
## do it only once

# from glob import glob
# import wave

# # pcm to wav and save to rDots_wav_dir

# def pcm_to_wav(audio_path, target_path, sampling_rate=16000):
#     if audio_path.split('.')[-1] != 'pcm':
#         raise BaseException('File is not pcm formatted')
    
#     with open(audio_path, 'rb') as pcmfile:
#         pcmdata = pcmfile.read()
#     with wave.open(audio_path.replace('pcm', 'wav'), 'wb') as wavfile:
#         wavfile.setparams((1, 2, sampling_rate, 0, 'NONE', 'NONE'))
#         wavfile.writeframes(pcmdata)

        
# pcmfiles = []
# pattern = '*.pcm'

# rDots_pcm_dir = os.path.join(rDots_home_dir, 'pcm/')

# rDots_wav_dir = os.path.join(rDots_home_dir, 'wav/')

# '''
# Copy substructure of pcm directory to new wav directory for the first run
# '''
# if not os.path.exists(rDots_wav_dir):
#     os.mkdir(rDots_wav_dir)

#     for dirpath, dirnames, filenames in os.walk(rDots_pcm_dir):
#         structure = os.path.join(rDots_wav_dir, dirpath[len(rDots_pcm_dir):])
#         if not os.path.isdir(structure):
#             os.mkdir(structure)

# for dir,_,_ in os.walk(os.path.join(rDots_home_dir, 'pcm/')):
#     pcmfiles.extend(glob(os.path.join(dir, pattern)))

# for pcmfile in pcmfiles:
#     pcm_to_wav(pcmfile, os.path.join(rDots_home_dir, 'wav/'))   

In [16]:
cnt = 0
data_dir = rDots_wav_dir
wav_files = {}
all_spks = []
for _, dirnames, _ in os.walk(data_dir):
    for dirname in dirnames:
        if os.path.isfile(os.path.abspath(dirname)): continue
        spk = dirname
        all_spks.append(spk)
        for filename in os.listdir(os.path.join(data_dir,dirname)):
            time, spk, sent = filename.rstrip('.wav').split('_')
            uniqID = filename.rstrip('.wav')
            wav_files[uniqID] = (spk, int(sent), filename)

In [18]:
df = pandas.DataFrame.from_dict(wav_files, 'index')

In [20]:
# set df's column names
df.columns = ['spk', 'sent', 'file']
df['set'] = ""

In [21]:
print("number of speakers: {}".format(len(all_spks)))

number of speakers: 64


In [22]:
new_col = [all_spks.index(x) for x in df.spk]
df = df.assign(label = new_col )

In [23]:
df.head()

,spk,sent,file,set,label
20150401220415579_m0045_40,m0045,40,20150401220415579_m0045_40.wav,,0
20150401220405767_m0045_32,m0045,32,20150401220405767_m0045_32.wav,,0
20150401220349980_m0045_36,m0045,36,20150401220349980_m0045_36.wav,,0
20150401220418653_m0045_17449,m0045,17449,20150401220418653_m0045_17449.wav,,0
20150401220403099_m0045_566,m0045,566,20150401220403099_m0045_566.wav,,0


### SV

In [119]:
# restrict speakers
chosen_spks = df.spk.value_counts().index[:100].tolist()

sv_test = df[(df.set == 'test') & (df.spk.isin(chosen_spks))]
sv_val = df[(df.set == 'val') & (df.spk.isin(chosen_spks))]
sv_train = df[(df.set == 'train') & (df.spk.isin(chosen_spks))]

In [154]:
len(google_spk)

30

In [155]:
# Ok google sentence
ok_speechs = df[df.sent == 32]
google_spk = list(ok_speechs.spk.value_counts().index[np.where(ok_speechs.spk.value_counts() > 10)])
# target_ok_speechs = ok_speechs[ok_speechs.spk.isin(google_spk)]

train_spk = google_spk[:20]
val_spk = google_spk[20:23]
test_spk = google_spk[23:]

sv_train = ok_speechs[ok_speechs.spk.isin(train_spk)]
sv_val = ok_speechs[ok_speechs.spk.isin(val_spk)]
sv_test = ok_speechs[ok_speechs.spk.isin(test_spk)]

import csv
import random
tags = ['train', 'val', 'test']
sets = {'train':sv_train, 'val':sv_val, 'test':sv_test}
for tag in tags:
    samples = []
    with open('sv_reddot_okgoogle_{}_manifest.csv'.format(tag), 'w') as f:
        for index, row in sets[tag].iterrows():
            file_path = os.path.join(data_dir, row.spk, row.file)
            label = google_spk.index(row.spk)          
            sample = ','.join([file_path, str(label+2)]) # 0,1 for speacial purpose
            samples.append(sample)
        random.shuffle(samples)
        writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
        writer.writerow(samples)

In [111]:
pretrain_speechs = df[(df.spk.isin(google_spk)) & ~(df.sent == 32)]

sv_train = pretrain_speechs.sample(frac=0.8,random_state=200)
sv_val = pretrain_speechs.drop(sv_train.index)
sv_test = sv_val.sample(frac=0.5)
sv_val = sv_val.drop(sv_test.index)

import csv
import random
tags = ['train', 'val', 'test']
sets = {'train':sv_train, 'val':sv_val, 'test':sv_test}
for tag in tags:
    samples = []
    with open('sv_reddot_pregoogle_{}_manifest.csv'.format(tag), 'w') as f:
        for index, row in sets[tag].iterrows():
            file_path = os.path.join(data_dir, row.spk, row.file)
            label = google_spk.index(row.spk)          
            sample = ','.join([file_path, str(label+2)]) # 0,1 for speacial purpose
            samples.append(sample)
        random.shuffle(samples)
        writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
        writer.writerow(samples)

In [125]:
# pairs for fine-tuning
def dfToPath(df):
    paths = []
    for _,row in df.iterrows():
        paths.append(os.path.join(row.spk, row.file))
    return paths

In [170]:
import itertools

val_speakers = test_spk
sub_df = sv_test
spk_pairs = itertools.combinations_with_replacement(val_speakers,2)
neg_audio_pair = []
pos_audio_pair = []
for spk_a, spk_b in spk_pairs:
    if spk_a != spk_b:
        audio_a = dfToPath(sub_df[sub_df.spk == spk_a].sample(n=5, replace=False))
        audio_b = dfToPath(sub_df[sub_df.spk == spk_b].sample(n=5, replace=False))
        neg_audio_pair = itertools.chain(neg_audio_pair, itertools.product(audio_a, audio_b))
    else:
        audio_a = dfToPath(sub_df[sub_df.spk == spk_a].sample(n=10, replace=False))
        pos_audio_pair = itertools.chain(pos_audio_pair, itertools.combinations(audio_a, 2))

In [171]:
samples = []
data_dir = "../../../sv_system/data"
root = rDots_wav_dir
with open(os.path.join(data_dir,'sv_reddot_pospair_manifest.csv'), 'w') as f:
    for (a, b) in pos_audio_pair:
        a_path = os.path.join(root,a)
        b_path = os.path.join(root,b)
        sample = ','.join([a_path,b_path,str(1)])
        samples.append(sample)
        random.shuffle(samples)
    writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
    writer.writerow(samples)

samples = []
with open(os.path.join(data_dir,'sv_reddot_negpair_manifest.csv'), 'w') as f:
    for (a, b) in neg_audio_pair:
        a_path = os.path.join(root,a)
        b_path = os.path.join(root,b)
        sample = ','.join([a_path,b_path,str(0)])
        samples.append(sample)
        random.shuffle(samples)
    writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
    writer.writerow(samples)              

In [1]:
df.to_pickle("/home/muncok/DL/dataset/SV_sets/Reddots_Dataframes.pkl")

NameError: name 'df' is not defined